In [47]:
import functools
import Numberjack

import forge
from data.logic import _util


Beth = 0
Charles = 1
David = 2
Frank = 3
Jessica = 4
Karen = 5
Taylor = 6

Weight = 0
Leight = 1
Whockey = 2
Lhockey = 3
asteroids = 4
baffle = 5
defender = 6
pacman = 7
qix = 8
invaders = 9

model = Numberjack.Model()

names = ['Beth', 'Charles', 'David', 'Frank', 'Jessica', 'Karen', 'Taylor']
n_names = list(range(len(names)))
activity_names = [
  'W 8ball', 'L 8ball',
  'W hockey', 'L hockey',
  'Asteroids', 'Baffle Ball', 'Defender', 'Pac-Man', 'Qix', 'Space Invaders',
]
people = list(range(len(names)))
rounds = list(range(6))
grid = []
for x in names:
  row = []
  grid.append(row)
  for y in rounds:
    row.append(Numberjack.Variable(0, len(activity_names) - 1, '%s_%s' % (x, y+1)))

units = {
  ' ': 0,
  'L': 1,
  'W': 2,
}
def parse_outcomes(s):
  result = {}
  for line in s.split('\n'):
    line = line.strip()
    if not line:
      continue
    lower, upper, spec = int(line[0]), int(line[1]), line[3:]
    acc = 0
    for i, c in enumerate(spec):
      acc += units[c] * (3 ** i)
    result[acc] = (lower, upper)
  return result

@functools.lru_cache()
def game_vector(n, g):
  return Numberjack.Sum(
    [grid[n][r] == g for r in rounds],
    coefs=[1, 2, 4, 8, 16, 32],
  )

@functools.lru_cache()
def game_count(n, g):
  return Numberjack.Cardinality(grid[n], g)

@functools.lru_cache()
def game_win_and_loss_vector(n, w, l):
  return Numberjack.Sum(
    [(grid[n][r] == w) for r in rounds] + [(grid[n][r] == l) for r in rounds],
    coefs=[2 * (3 ** r) for r in rounds] + [1 * (3 ** r) for r in rounds],
  )

# Setup.
# Eight-ball and hockey are always occupied. Other things are optional, limit 1.
for r in rounds:
  model.add(Numberjack.Gcc(
    [grid[n][r] for n in range(len(names))],
    {
      Weight: (1, 1),
      Leight: (1, 1),
      Whockey: (1, 1),
      Lhockey: (1, 1),
      asteroids: (0, 1),
      baffle: (0, 1),
      defender: (0, 1),
      pacman: (0, 1),
      qix: (0, 1),
      invaders: (0, 1),
    }
  ))
# No one played the same machine twice (or won more than 3 games).
for n in range(len(names)):
  model.add(Numberjack.Gcc(
    grid[n],
    {
      Weight: (0, 3),
      Leight: (0, 1),
      Whockey: (0, 3),
      Lhockey: (0, 1),
      asteroids: (0, 1),
      baffle: (0, 1),
      defender: (0, 1),
      pacman: (0, 1),
      qix: (0, 1),
      invaders: (0, 1),
    }
  ))
# The tournament is single elimination. That means:
# 1. Every round has (win + loss)*2. Implemented with first GCC.
# 2. There are 12 total plays over 6 matches. Implemented above.
# 3. "In the first 3 periods 3 different pairs of people played..."
#    Implies each player can only play 1 <game> in first 3 rounds.
#for n in range(len(names)):
#  model.add(Numberjack.Sum([
#    [(grid[n][r] == Weight) | (grid[n][r] == Leight) for r in rounds[:3]]
#  ]) == 1)

#    123456
# 11 L
# 01 W  L
# 01 W  W L
# 01 W  W W
# 01    L
# 01    W L
# 01    W W
# 11  L
# 01  W  L
# 01  W  WL
# 01  W  WW
# 11   L
# 01   W L
# 01   W WL
# 01   W WW
# 4. First loss is last game.
outcomes = parse_outcomes("""
  11 L
  01 W  L
  01 W  W L
  01 W  W W
  01    L
  01    W L
  01    W W
  11  L
  01  W  L
  01  W  WL
  01  W  WW
  11   L
  01   W L
  01   W WL
  01   W WW
""")
if False:
  model.add(Numberjack.Gcc(
    [game_win_and_loss_vector(n, Weight, Leight) for n in range(len(names))],
    outcomes,
  ))
  model.add(Numberjack.Gcc(
    [game_win_and_loss_vector(n, Whockey, Lhockey) for n in range(len(names))],
    outcomes,
  ))

#for n in range(len(names)):
  #model.add((
  #  Numberjack.Conjunction([grid[n][r] != Leight for r in rounds]) |
  #  (game_vector(n, Leight) > game_vector(n, Weight))
  #))
  #model.add((
  #  Numberjack.Conjunction([grid[n][r] != Lhockey for r in rounds]) |
  #  (game_vector(n, Lhockey) > game_vector(n, Whockey))
  #))

#1a: One person only played Qix.
Numberjack.Sum(
  [game_count(n, qix) == 1]
)


solver = model.load('MiniSat')
print(str(model))
solver.solve()

if solver.is_unsat():
  print('Impossible')
else:
  for i, n in enumerate(names):
    print(n, _util.numberjack_solution(game_win_and_loss_vector(i, Weight, Leight)))
  print('\t'.join(['name', '1', '2', '3', '4', '5', '6']))
  for y, row in enumerate(grid):
    result = [names[y]]
    for x, variable in enumerate(row):
      value = variable.get_value()
      result.append(activity_names[value][:7])
    print('\t'.join(result))

assign:
  Beth_1 in {0..9}
  Charles_1 in {0..9}
  David_1 in {0..9}
  Frank_1 in {0..9}
  Jessica_1 in {0..9}
  Karen_1 in {0..9}
  Taylor_1 in {0..9}
  Beth_2 in {0..9}
  Charles_2 in {0..9}
  David_2 in {0..9}
  Frank_2 in {0..9}
  Jessica_2 in {0..9}
  Karen_2 in {0..9}
  Taylor_2 in {0..9}
  Beth_3 in {0..9}
  Charles_3 in {0..9}
  David_3 in {0..9}
  Frank_3 in {0..9}
  Jessica_3 in {0..9}
  Karen_3 in {0..9}
  Taylor_3 in {0..9}
  Beth_4 in {0..9}
  Charles_4 in {0..9}
  David_4 in {0..9}
  Frank_4 in {0..9}
  Jessica_4 in {0..9}
  Karen_4 in {0..9}
  Taylor_4 in {0..9}
  Beth_5 in {0..9}
  Charles_5 in {0..9}
  David_5 in {0..9}
  Frank_5 in {0..9}
  Jessica_5 in {0..9}
  Karen_5 in {0..9}
  Taylor_5 in {0..9}
  Beth_6 in {0..9}
  Charles_6 in {0..9}
  David_6 in {0..9}
  Frank_6 in {0..9}
  Jessica_6 in {0..9}
  Karen_6 in {0..9}
  Taylor_6 in {0..9}
  
subject to:
   Gcc(Beth_1 Charles_1 David_1 Frank_1 Jessica_1 Karen_1 Taylor_1 | 0 in [1,1] 1 in [1,1] 2 in [1,1] 3 in [1,1] 

In [39]:
units = {
  ' ': 0,
  'L': 1,
  'W': 2,
}
def parse_outcomes(s):
  result = {}
  for line in s.split('\n'):
    line = line.strip()
    if not line:
      continue
    lower, upper, spec = int(line[0]), int(line[1]), line[3:]
    acc = 0
    for i, c in enumerate(spec):
      acc += units[c] * (3 ** i)
    result[acc] = (lower, upper)
  return result

parse_outcomes("""
  11 L
  01 W  L
  01 W  W L
  01 W  W W
  01    L
  01    W L
  01    W W
  11  L
  01  W  L
  01  W  WL
  01  W  WW
  11   L
  01   W L
  01   W WL
  01   W WW
""")

{1: (1, 1),
 3: (1, 1),
 9: (1, 1),
 27: (0, 1),
 29: (0, 1),
 87: (0, 1),
 99: (0, 1),
 297: (0, 1),
 299: (0, 1),
 411: (0, 1),
 423: (0, 1),
 540: (0, 1),
 542: (0, 1),
 654: (0, 1),
 666: (0, 1)}